In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import Activation, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tf_utils.callbacks import schedule_fn2
from tf_utils.Cifar10DataAdvanced import CIFAR10

In [2]:
IMAGENET_SIZE = 96
IMAGENET_DEPTH = 3
IMAGENET_SHAPE = (IMAGENET_SIZE, IMAGENET_SIZE, IMAGENET_DEPTH)

In [3]:
def build_model(img_shape, num_classes) -> Model:
    base_model = MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=IMAGENET_SHAPE
    )

    num_layers = len(base_model.layers)
    print(f"Number of layers in the base model: {num_layers}")
    fine_tune_at = num_layers - 10
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    input_img = Input(shape=img_shape)
    x = Rescaling(scale=2.0, offset=-1.0)(input_img)
    x = Resizing(height=IMAGENET_SIZE, width=IMAGENET_SIZE)(x)
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs=[input_img],
        outputs=[y_pred]
    )

    model.summary()

    return model

- Best model from chapter 8:   0.7200 accuracy
- Best model from chapter 9_3: 0.8361 accuracy
- Best model from chapter 9_7: 0.8993 accuracy

In [4]:
data = CIFAR10()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

In [5]:
img_shape = data.img_shape
num_classes = data.num_classes

In [6]:
epochs = 100

In [ ]:
model = build_model(
    img_shape,
    num_classes
)

opt = Adam(learning_rate=5e-4)

model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

lrs_callback = LearningRateScheduler(
    schedule=schedule_fn2,
    verbose=1
)

es_callback = EarlyStopping(
    monitor="val_loss",
    patience=30,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
model.fit(
    train_dataset,
    verbose=1,
    epochs=epochs,
    callbacks=[lrs_callback, es_callback],
    validation_data=val_dataset,
)

In [ ]:
scores = model.evaluate(
    val_dataset,
    verbose=0
)
print(f"Scores: {scores}")